# **BEFORE YOU START**

*  Click on the dropdown arrow next to the "RAM DISK" metric on the top right
*  Click "change runtime type"
*  Select "T4 GPU"

You can run this without doing these steps, but it will be very slow





# Import necessary libraries

In [1]:
import torch
from torch import nn, save, load
from torch.optim import Adam
import torchvision
from torchvision import transforms

# Data Loading & Preprocessing

In [2]:
#this is for data preprocessing and loading with train data
def train_pl():
    #the transformation we will apply to the images from the FER2013 dataset
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor(), # Convert image to tensor
        transforms.Normalize(0.485, 0.229) # Normalize image
    ])

    # loading the data from the directory I have stored the downloaded FER2013 dataset
    train_data = torchvision.datasets.FER2013(root='/content/dataset', split = 'train', transform=transform)
    print(f"Length of train data: {len(train_data)}")
    # create dataloaders so that the FER2013 data can be loaded into the model we will implement
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=19, shuffle=True, num_workers=2)

    return train_loader

#this is for data preprocessing and loading with test data
def test_pl():
    #the transformation we will apply to the images from the FER2013 dataset
    transform = transforms.Compose([
        # transforms.Grayscale(),
        transforms.ToTensor(), # Convert image to tensor
        transforms.Normalize(0.485, 0.229) # Normalize image
    ])

    # loading the data from the directory I have stored the downloaded FER2013 dataset
    test_data = torchvision.datasets.FER2013(root='/content/dataset', split = 'test' ,  transform=transform)
    print(f"Length of test data: {len(test_data)}")
    # create dataloaders so that the FER2013 data can be loaded into the model we will implement
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False, num_workers=2)

    return test_loader

# Designing the model

In [3]:
class EmotionModel(nn.Module):
    def __init__(self):
        super(EmotionModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 6 * 6, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 7)  # 7 classes for different emotions
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


# Training the model

In [4]:
def train():
    model = EmotionModel().to('cuda')
    optimizer = Adam(model.parameters(), lr = 1e-3)
    loss_fn = nn.CrossEntropyLoss()

    train_set = train_pl()

    for epoch in range(50): #train for 50 epochs
        for batch in train_set:
            X, y = batch
            X, y = X.to('cuda'), y.to('cuda')
            prediction = model(X)
            loss = loss_fn(prediction, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}\n-------------------------------")
        print(f"\tloss:{loss}")
        print("--------------------------------------------")

    # saving our model to our environment

    return model

# Tying everything together to have a savable model

In [ ]:
    trained_model = train()
    save(trained_model, 'model_MK1')

# Code to run the model on images
### **Don't worry about this code. You aren't required to understand or learn this for the purposes of making a deep learning model. This is just for demonstration of the model**

In [5]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.6 MB/s eta 0:00:00


In [6]:
import cv2
import torch
import numpy as np
from mtcnn import MTCNN
from torchvision import transforms
from google.colab.patches import cv2_imshow

def run_model(input_image_path, output_image_path):
  # Load trained model
  model = torch.load('/content/model_MK1')
  model.eval()
  model.to(torch.device('cuda'))
  model = torch.jit.script(model)

  # Load emotion labels
  emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

  # Load and preprocess the input image
  input_image = cv2.imread(input_image_path)
  gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

  # Initialize MTCNN for face detection
  mtcnn = MTCNN()

  # Detect faces in the image
  faces = mtcnn.detect_faces(input_image)

  for face_info in faces:
      x, y, w, h = [int(coord) for coord in face_info['box']]
      face = gray_image[y:y + h, x:x + w]

      # Preprocess the face image
      face = cv2.resize(face, (48, 48))
      face_tensor = transforms.ToTensor()(face).unsqueeze(0).to(torch.device('cuda'))

      with torch.no_grad():
          predictions = model(face_tensor)
      predicted_emotion = emotion_labels[predictions.argmax()]

      cv2.rectangle(input_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.putText(input_image, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

  # Display or save the output image
  cv2.imwrite(output_image_path, input_image)
  cv2_imshow(input_image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

## Run the model on a screenshot of **happy** face stock images

In [ ]:
run_model('/content/test_images/happy_test.png', '/content/results/happy_result.png')

## Run the model on a screenshot of **sad** face stock images


In [ ]:
run_model('/content/test_images/sad_test.png', '/content/results/sad_result.png')

## Run the model on a screenshot of **angry** face stock images

In [ ]:
run_model('/content/test_images/angry_test.png', '/content/results/angry_result.png')

## Run the model on a screenshot of **fear** face stock images

In [ ]:
run_model('/content/test_images/fear_test.png', '/content/results/fear_result.png')

## Run the model on a screenshot of **surprised** face stock images

In [ ]:
run_model('/content/test_images/surprised_test.png', '/content/results/surprised_result.png')

## Run the model on a screenshot of **disgusted** face stock images

In [ ]:
run_model('/content/test_images/disgusted_test.png', '/content/results/disgusted_result.png')

## Run the model on a screenshot of **neutral** face stock images

In [ ]:
run_model('/content/test_images/neutral_test.png', '/content/results/neutral_result.png')